In [ ]:
!pip install neuralcoref
!pip install torch torchvision torchaudio --quiet
!pip install transformers[torch] --quiet
!pip install accelerate --quiet
!pip install accelerate>=0.20.1 --quiet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import re

# text = "John: \"Alicia, Please schedule me to attend this meeting.\""
# text = re.sub(r'\b\w+:', '', text).replace("\"", "").strip()
# print(text)

Alicia, Please schedule me to attend this meeting.


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import re

# Load data from a text file, assuming one sentence per line with a tab character separating the sentence from the label
def load_data(file_path):
    texts, labels = [], []
    with open(file_path, 'r') as file:
        for line in file:
            label, text = line.strip().split('\t')
            text = re.sub(r'\b\w+:', '', text).replace("\"", "").strip()
            texts.append(text)
            labels.append(0 if label == 'NORMAL' else 1)  # Assigning 'NORMAL' to 0 and 'ACTION' to 1
    return texts, labels

text, label = load_data('/content/drive/MyDrive/reu2023/action_item_classification/action_item_dataset.txt')
print(text)


# Preprocess the data
class ActionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



In [ ]:
# Loading the data
texts, labels = load_data('/content/drive/MyDrive/reu2023/action_item_classification/action_item_dataset.txt') # Replace with your file path
for i in range(20):
  texts.append('next meeting will be')


for i in range(20):
  labels.append(1)
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)



# Tokenize the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_text_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=92)
test_text_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=92)

# Find ideal max_length
'''
def truncate_zeros(lst):
    while lst[-1] == 0:
        lst = lst[:-1]
    return lst
lengths = []
for text_tokenization in train_text_encodings['input_ids']:
  #print(len(truncate_zeros(text_tokenization)))
  lengths.append(len(truncate_zeros(text_tokenization)))
#print(encoding_max_length)
'''

# Create PyTorch Dataset
train_dataset = ActionDataset(train_text_encodings, train_labels)
test_dataset = ActionDataset(test_text_encodings, test_labels)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/reu2023/action_item_classification/results2',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/reu2023/action_item_classification/logs2',
)



In [ ]:
# Load the pre-trained BERT model and train
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

training_output = trainer.train()
print('training: ', training_output)

# Evaluate the model
eval_output = trainer.evaluate()
print('eval: ', eval_output)

#model.save_pretrained('/content/drive/MyDrive/reu2023/action_item_classification/models')
trainer.save_model('/content/drive/MyDrive/reu2023/action_item_classification/models3')







In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')  # Download the punkt tokenizer if not already downloaded

text_ex = '''

"Speaker A: Okay. Well this is what me and Richard came up with. The default spot for the on-off button. The mute button just below that. Then there\'s the volume and channel selectors. Simple plus-minus button. Uh we thought of a help button. If you hold a help button and you press another button, uh the help goes to the L_C_D_ screen. Then there\'s the zero to one to zero buttons. A button for teletext. A button for the subtitles. And the company logo. So it\'s rather simple prototype. And uh we\'ll have to see from testing uh how the users take it. Sure uh just pop in at any time. I think if you look at most controls, they\'ve got more buttons than this. And well the on-off button, it\'s it\'s a necessity. You can\'t drop that one. The volume and channel buttons, you need you obviously need those those. A button for teletext could be replaced uh by pressing the volume-down button twice. So we could cancel that one. I think A button for teletext really is necessary because there\'s no other way to know when someone wants to know what a button does. Well, you can\'t leave out the number buttons I guess. And Yeah. So it\'s rather basic already. Yeah, they\'re It\'s rather hard to draw on the white-board. But they\'re supposed to be equal sized, round, with a with a little logo on it for the for the volume, the the triangle and stuff. Just for recognition. So and for the materials we\'ve just chosen for rubber buttons. With a a different colour than the case. So they jump out. And uh that\'s about it. Yep. Yes. Yep. Yep. Yeah. Yeah. Yes. Uh that\'s that\'s a big cost. If we leave out the display we can also save money on the chip. Which isn\'t the the docking station isn\'t even in this c s schematic. So the the docking station\'s not even taken into the price. we we could save money on the the docking station. But would the the docking station make the remote more usable for elderly people? My mother can\'t even send send an S_M_S_ message. So Yeah. The margin will get too small. Yeah. Ten. No remote. That\'s we\'re getting closer. Yeah. You could just not scroll for a half a second. So you won\'t need a button. If you want to use an L_C_D_ screen you you need an advanced chip, yes. Yep. Yeah if you v could just leave out one more button. Yeah. I think so. Yeah. What if you have to scroll to page eighty eight? Eight hundred eighty eight. Yeah. Okay. No. we could make two buttons out of that. And just um If you press the volume button you can control the volume with the scroll-wheel. So that would save two buttons. If you do the same for the channel. More obvious. So if we Yeah. So If we leave out all those buttons, perhaps you can go with the flat flat case. And make the flat flat case smaller overall. So if you put the the volume and channel buttons on the same height as the on-off button, the screen right behi under that, than the scroll button you get a a much smaller remote. And the scroll button you get a a much smaller remote sh Yeah? Yeah. Richard\'s argument was very good. Yeah. Yep. Yeah. Six. Yeah. Four or five. Yes. No. Mm, we haven\'t thought of that one. I think that\'s a three. I still I think it\'s too m too fancy. Too too flashy. Twendag sieven an twendag. Yeah. Yes. Yeah. That\'s true. Yeah, there w there was enough room, but the finance uh Yes. Yes, was okay. The tasks are very structured, so you can just do step by step and it\'s very easy. Well, the smart-board, the d d drawing is just a pain in the ass, but The digital the digital pen is very nice. Yes. Yeah. A flip-over or a more precise uh digit Yes. No. Didn\'t work. Yes. Yeah. Considering we are not going to make a docking station. Perhaps. Mm-hmm. Okay. Okay. Speaker B: Good afternoon. So Hello. No problem. Okay. That\'s possible on uh this time of day. Starts at three o\'clock. Uh Uh, let me see. we fourth and last meeting. There he is. Yes. Okay this we last meeting. In Our fourth and last meeting we will discuss we final design. And we will do some evaluation about the, not only the product, but also the project. And then we\'re going to close the project today as well. So after this you will be uh free to go and uh spend all your money. And our final design, detailed design meeting uh we will discuss uh the look-and-feel of our final design, our final design, and we will do the product evaluation. Um, in order to do that we have this agenda. we\'ll have the prototype presentation first. Then we will set up some evaluation cri um criteria. Then we will look at the finance. Uh we will have to see if everything we wanna do is also possible within we budget. Because everything costs money, and the more functions you wanna have the more money it will cost. So we have to see if it fits within the twelve and a half Euro per remote. But we will see that later. Then we will do the product evaluation, and the closing after that. we\'ve got forty minutes. So we will be finished at half past three. But first um we will do the this is prototype presentation. So, if some yeah if somebody wants step forward. Okay. Okay. Uh but um I thought in a few meetings earlier we uh uh tried to keep the this uh simple. Uh just a few buttons and large buttons. But uh I think these are altogether quite a quite a few buttons. So I\'m wondering if we if we neely really need all of quite a quite a few buttons. That\'s right. Yeah. Yeah. Mm. Long time. Yeah. If you put Yeah. Okay. Yeah. Uh teletext and subtitles are yeah necessary. Okay. Yep. Okay. So we had somebody about th interfa Something about the design of the buttons there? Just n normal plain buttons. Yeah. \'Kay. No. No. Yeah. Okay. Yep. Okay. That\'s nice. Then because we only have thirty minutes left, I will move on to the finance part, which is pretty exciting, to see if it\'s all possible what we wanna do. And I can tell you that we\'re going to have a pretty hard time producing this for twelve and a half Euros. If we see I don\'t know if I\'ve filled in correctly, so just correct me if you see uh something wrong. I counted two batteries. But maybe we can also use one. I don\'t know if it\'s possible. it\'s possible\'s rechargeable. That\'s right. Okay. That\'s two Euros off. We need the advanced chip. So there\'s not much to uh to save there. Here We have the single curves. Uh We can see that the difference between uncurved and the single curved is one Euro. Um, I don\'t know, but I think the single curved is good for design, and also for the display to have a prominent place on the remote control. I think We have to keep that. Then We have the case material supplements. It\'s plastic. It\'s the cheapest one we need. So that\'s uh not much to save either. But then the biggest costs are the buttons. So maybe We really should try to discard some buttons to uh keep the biggest costs low. Because you have to We will have to get the twelve and a half Euros at the end. Um, so if We LetWe fir first count the buttons We have now. Because I Sixteen. Nah, that will be even more then. Eighteen Euros. So, seventeen. Okay, including the help? Yeah. Uh seventeen. I think We can uh discard the help and the mute button uh by pressing down volume long, or pressing down a a number long. That saves We uh one Euro already. \'Cause then We have got fifteen I think? Fifteen buttons. And this is No those are one, I think. Yeah but th it\'s not stated in this files. So maybe We can We can even make one button with the volume and the channel in one, by pressing That\'s possibility as well. So And it\'s good for the design as well. So you can make Uh let\'s see. If you make this Looks a bit like uh a cross. Plus. Min. Uh s yeah channel. Yeah w Yeah th I think they count uh the materials. Yeah on the chip there. Yeah. That\'s right. That\'s right. But I think because we have the advanced chip we can just count this as one button. But No but I think Yeah. Maybe it is, but then it seems to Min that it\'s impossible to get the twelve and a half Euros. Also the L_C_ display, I think it\'s, I think it\'s too expensive for the display we use. I think they uh try we That\'s right, but what\'s the big advantage of our remote then? Yeah. No. That\'s extra. That\'s extra. That\'s right. That\'s an option. Poland. Something. Polish supplier. Don\'t you think we can, if we can count this as v as one button, and integrate th uh these buttons in three, then we save a lot of money as well. Yeah. That\'s a point. Yep. No. Yeah but uh we have to stick to the twelve and a half Euros. we don\'t have any more budget to develop it. Yeah. So It is. If you leave out the L_C_ display. And if you use less buttons. Say Or you can take the single chip. There it is. But then w Good looking. Yeah. I think it\'s uh difficult as well, but Or we can leave out the ten buttons and take one scroll wheel for the programme numbers. Then w Because then we save ten buttons. Then we have five and one. And and see. If we have this one and we\'ve got Then w uh, we\'re getting close. Then you will Or maybe you can um scroll. If you scroll you will see the numbers on the L_C_D_ display. Until you\'ve got the right number, then you push the right number. Alright. It\'s gotta scroll and push. Okay but then you you can push uh some other button as well. Yeah. That\'s right. So if you scroll to a number and then you wait a half second, then a number g turns to that channel. But a number would definitely crop cost, a lot. And we need the battery. And the regular chip is not possible? It has to be advanced? It has to be advanced. Yeah. Okay. And we can save a Euro by a flat design. That\'s an option we can Then we\'re almost there. Yeah. So if we Yeah but I think that\'s That is a big advantage, if we\'re But Can we use can\'t we integrate the teletext and the subtitles in one button? If you push it three times? Yeah. Ah that\'s not really that No. I think if you make a good advertisement uh on television and in the in the guide, you can explain to the people how to use the scroll wheel. If you just make it real simple. Because it saves it saves a lot of money. And we can keep we L_C_D_ screen, which can provide extra information. How to use the scroll wheel. How to use the other bu buttons as subtitles. And it\'s good for the innovative design as well. If you would erase these. Mm eraser? And we put uh Looks a bit odd maybe. That is Something like that. Then, yeah. we\'ve got the scroll wheel. One, two, three, four, five buttons, if we erase this one. And these are two buttons then. Yeah. Yeah. That\'s really a good good idea, I think. And it will make the use of the scroll wheel uh more obvious indeed. So we make one for the volume, one for the channel. Plus scroll. That\'s right. So we\'ve got one, two, three. Yeah, we can leave the teletext in if we want. That\'s m that\'s better. Y yeah. Mm yeah. Uh yeah, and then you can That\'s right. So we can decrease this one to four buttons. Yeah. That\'s no problem. Yeah. Yeah. It\'s uh It\'s more the moulds in which they are being made, I think. Single curved is really easy to just fill in. And cases come right out of the machine. And single curved you have to have a different uh different machine, than a flat one. I think this is a really easy one. Um, what does everybody think about a flat or a single curved design? Freek, what do you think? Yeah. Yep. We have to cut costs. Yeah. Yeah I\'m ag Ask for more money. Yeah. Yeah. I am agreeing with the usability. Yeah. Yeah. But th um I think We can then keep the single curved one. Just to express We L_C_D_ screen a bit more. So people will use our L_C_D_ screen more, and especially for the help functions, it will be good if we have scroll bar, scroll button. It will be necessary to have good help function, as well. So this is scroll. I think it was better to have this price list a bit earlier. Before We went on to the to the whole design. But I\'m glad We could make a bit. It\'s pretty different, if you saw the last one than this one. Yeah. It will. Yeah. No, but I think I think the most That\'s right.

'''


#sentences = sent_tokenize(text_ex)

# 1. Load the model and the tokenizer
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/reu2023/action_item_classification/models')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def action_item_extraction(text):
  index = -1
  actions = []
  surroundings = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    index+=1
    # next meeting case
    if 'next meeting' in sentence:
      actions.append((sentence, index))
      continue

    # Tokenize the text
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

    # 3. Feed the tokenized text into the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Raw output to probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # 4. Interpret the model's output
    _, predicted_class = torch.max(probabilities, dim=1)

    # print(f'Text: {text}')
    # print(f'Class probabilities: {probabilities}')
    # print(f'Predicted class: {predicted_class.item()}')
    #print(sentence, predicted_class.item())
    if predicted_class.item() == 1:
      actions.append((sentence, index))
      surrounding = ''
      surrounding += sentences[index - 3]
      surrounding += sentences[index - 2]
      surrounding += sentences[index - 1]
      surrounding += sentences[index]
      if index == len(sentences) - 1:
        surroundings.append(surrounding)
      else:
        surrounding += sentences[index + 1]
        surrounding += sentences[index + 2]
        surroundings.append(surrounding)

  return actions, surroundings



print(action_item_extraction(text_ex))
print(len(action_item_extraction(text_ex)))






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


([('Sure uh just pop in at any time.', 14), ('And make the flat flat case smaller overall.', 73), ('Uh Uh, let me see.', 117), ("So you can make Uh let's see.", 209), ('If you leave out the L_C_ display.', 242), ('Say Or you can take the single chip.', 244), ('And and see.', 251), ('How to use the other bu buttons as subtitles.', 281), ('Plus scroll.', 296)], ["And the company logo.So it's rather simple prototype.And uh we'll have to see from testing uh how the users take it.Sure uh just pop in at any time.I think if you look at most controls, they've got more buttons than this.And well the on-off button, it's it's a necessity.", 'More obvious.So if we Yeah.So If we leave out all those buttons, perhaps you can go with the flat flat case.And make the flat flat case smaller overall.So if you put the the volume and channel buttons on the same height as the on-off button, the screen right behi under that, than the scroll button you get a a much smaller remote.And the scroll button you get 

In [ ]:
#EVALUATE
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
    }

